In [1]:
using Pkg
Pkg.activate("Phase2_projet")
Pkg.add(["LinearAlgebra", "SparseArrays", "Krylov", "BenchmarkTools", "SuiteSparseMatrixCollection", "MatrixMarket"])
using LinearAlgebra, SparseArrays, Krylov, BenchmarkTools, SuiteSparseMatrixCollection, MatrixMarket, Random

  Activating project at `c:\Users\ulric\Projet_MTH8211\Phase2_projet`
   Resolving package versions...
  No Changes to `C:\Users\ulric\Projet_MTH8211\Phase2_projet\Project.toml`
  No Changes to `C:\Users\ulric\Projet_MTH8211\Phase2_projet\Manifest.toml`
[ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf] (cache misses: wrong dep version loaded (4))


In [2]:
println("Nombre de threads actuellement activés : ", Threads.nthreads())

Nombre de threads actuellement activés : 16


In [14]:
function badly_conditioned_rectangular_matrix(m, n, kappa)
    U, _ = qr(randn(m, n))
    V, _ = qr(randn(n, n))
    s = range(1.0, 1.0/kappa, length=n)
    S = Diagonal(s)
    A = U * S * V'
    return Matrix(A)
end

badly_conditioned_rectangular_matrix (generic function with 1 method)

In [15]:
function badly_conditioned_underdetermined_matrix(m, n, kappa)
    U, _ = qr(randn(n, m))
    V, _ = qr(randn(m, m))
    s = range(1.0, 1.0 / kappa, length=m)
    S = Diagonal(s)
    A_tall = U * S * V'
    return Matrix(A_tall')[1:m, 1:n]
end

badly_conditioned_underdetermined_matrix (generic function with 1 method)

In [7]:
function lsrn_lsqr(A, b; gamma=2.0, tol=1e-10, itmax=2000)
    m, n = size(A)
    s = ceil(Int, gamma * n) 

    G = randn(s, m)

    Ã = G * A

    Ũ, Σ̃, Ṽ = svd(Ã; full=false)
    r = sum(Σ̃.> 1e-12)

    Σinv = Diagonal(1.0 ./ Σ̃[1:r])
    V_r = Ṽ[:,1:r]
    N = V_r * Σinv

    AN = A * N
    ŷ, histo = lsqr(AN, b; atol=tol, btol=tol, itmax=itmax, history=true)

    x̂ = N * ŷ
    return x̂, histo
end

lsrn_lsqr (generic function with 1 method)

In [10]:
function lsrn_lsqr_underdetermined(A, b; gamma=2.0, tol=1e-10, itmax=2000)
    m, n = size(A)
    s = ceil(Int, gamma * m)
    G = randn(n, s)
    Ã = A * G

    Ũ, Σ̃, Ṽ = svd(Ã; full=false)
    r = sum(Σ̃ .> 1e-12)
    U_r = Ũ[:,1:r]
    Σ_r = Σ̃[1:r]

    Σinv = Diagonal(1.0 ./ Σ_r)
    M = U_r * Σinv      

    Mt = M'
    At_pre = Mt * A    
    bt_pre = Mt * b     

    x̂, histo = lsqr(At_pre, bt_pre; atol=tol, btol=tol, itmax=itmax, history=true)

    return x̂, histo
end

lsrn_lsqr_underdetermined (generic function with 1 method)

In [25]:
m = 100000
n = 1000
kappa = 1e9
A = badly_conditioned_rectangular_matrix(m, n, kappa)
x = randn(n)
b = A * x
println("Conditionnement de A :", cond(A))

Conditionnement de A :9.999999996072491e8


In [13]:
BLAS.get_num_threads()

16

In [26]:
@benchmark lsrn_lsqr(A, b; gamma=2.0, tol=1e-10, itmax=2000) 

BenchmarkTools.Trial: 2 samples with 1 evaluation per sample.
 Range (min … max):  3.908 s …    4.313 s  ┊ GC (min … max): 1.03% … 1.77%
 Time  (median):     4.110 s               ┊ GC (median):    1.42%
 Time  (mean ± σ):   4.110 s ± 286.480 ms  ┊ GC (mean ± σ):  1.42% ± 0.52%

  █                                                        █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  3.91 s         Histogram: frequency by time         4.31 s <

 Memory estimate: 2.33 GiB, allocs estimate: 79.

In [27]:
@benchmark lsqr(A, b; atol=1e-10, btol=1e-10, itmax=2000, history=true)

BenchmarkTools.Trial: 1 sample with 1 evaluation per sample.
 Single result which took 34.445 s (0.00% GC) to evaluate,
 with a memory estimate of 1.60 MiB, over 42 allocations.

In [29]:
# LSQR 
println("\n--- LSQR sans préconditionneur ---")
res1, hist1 = lsqr(A, b; atol=1e-10, btol=1e-10, itmax=2000, history=true)
println("Itérations LSQR (sans préc)      : ", hist1.niter)
println("Résidu final LSQR (sans préc)    : ", norm(A * res1 - b) / norm(b))

#lsqr-lsrn
println("\n--- LSQR avec préconditionneur LSRN ---")
x_prec, hist2 = lsrn_lsqr(A, b; gamma=2.0, tol=1e-10, itmax=2000)
println("Itérations LSQR (LSRN)           : ", hist2.niter)
println("Résidu final LSQR (LSRN)         : ", norm(A * x_prec - b) / norm(b))

println("\n--- Résumé du nombre d'itérations ---")
println("LSQR sans préc           : $(hist1.niter)")
println("LSQR avec préc LSRN      : $(hist2.niter)")



--- LSQR sans préconditionneur ---
Itérations LSQR (sans préc)      : 1501
Résidu final LSQR (sans préc)    : 3.4338826909057485e-7

--- LSQR avec préconditionneur LSRN ---
Itérations LSQR (LSRN)           : 43
Résidu final LSQR (LSRN)         : 1.7620724051574278e-7

--- Résumé du nombre d'itérations ---
LSQR sans préc           : 1501
LSQR avec préc LSRN      : 43


In [ ]:
m = 1000   
n = 10000  
kappa = 1e9
A = badly_conditioned_underdetermined_matrix(m, n, kappa)
x_exact = randn(n)
b = A * x_exact

println("Conditionnement estimé (A*A') :", cond(A * A'))

# LSQR direct (va donner une solution quelconque, pas forcément de norme minimale)
println("\n--- LSQR standard (sous-déterminé) ---")
res1, hist1 = lsqr(A, b; atol=1e-10, btol=1e-10, itmax=2000, history=true)
println("Itérations LSQR : ", hist1.niter)
println("Résidu final LSQR : ", norm(A * res1 - b) / norm(b))

# LSRN + LSQR
println("\n--- LSRN (LSQR, sous-déterminé) ---")
x_lsrn_lsqr, hist3 = lsrn_lsqr_underdetermined(A, b; gamma=2.0, tol=1e-10, itmax=2000)
println("Itérations LSQR (LSRN) : ", hist2.niter)
println("Résidu LSRN (LSQR) : ", norm(A * x_lsrn_lsqr - b) / norm(b))



println("\n--- Résumé du nombre d'itérations ---")
println("LSQR standard         : $(hist1.niter)")
println("LSRN LSQR             : $(hist2.niter)")

Conditionnement estimé (A*A') :1.1271368085804024e17

--- LSQR standard (sous-déterminé) ---
Itérations LSQR : 1491
Résidu final LSQR : 6.446143442607677e-7

--- LSRN (LSQR, sous-déterminé) ---
Itérations LSQR (LSRN) : 43
Résidu LSRN (LSQR) : 2.354657661411113e-7

--- Résumé du nombre d'itérations ---
LSQR standard         : 1491
CG standard           : 131
LSRN LSQR             : 43
LSRN CG               : 44


### CODE NON TERMINÉ OU NON UTILISÉ

In [ ]:
BLAS.set_num_threads(1)

In [ ]:
function lsrn_cg(A, b; gamma=2.0, tol=1e-10, itmax=2000)
    m, n = size(A)
    s = ceil(Int, gamma * n)
    G = randn(s, m)
    Ã = G * A

    # SVD compacte
    Ũ, Σ̃, Ṽ = svd(Ã; full=false)
    r = sum(Σ̃ .> 1e-12)
    Σinv = Diagonal(1.0 ./ Σ̃[1:r])
    V_r = Ṽ[:,1:r]
    N = V_r * Σinv

    # Forme normale
    AN = A * N
    AtAN = AN' * AN
    AtbN = AN' * b

    # Résoudre par CG
    ŷ, histo = cg(AtAN, AtbN; atol=tol, rtol=tol, itmax=itmax, history=true)

    # Reconstruire la solution min-norme
    x̂ = N * ŷ
    return x̂, histo
end

In [ ]:
function lsrn_lsqr_parallel(A, b; gamma=2.0, tol=1e-10, itmax=2000)
    m, n = size(A)
    s = ceil(Int, gamma * n)
    G = randn(s, m)

    B = zeros(s, n)

    nthreads = Threads.nthreads()
    blocksize = ceil(Int, s / nthreads)

    Threads.@threads for t = 1:nthreads
        i1 = (t-1)*blocksize + 1
        i2 = min(t*blocksize, s)
        if i1 <= i2 
            B[i1:i2, :] .= G[i1:i2, :] * A
        end
    end

    Ũ, Σ̃, Ṽ = svd(B; full=false)
    r = sum(Σ̃ .> 1e-12)
    Σinv = Diagonal(1.0 ./ Σ̃[1:r])
    V_r = Ṽ[:, 1:r]
    N = V_r * Σinv

    AN = A * N
    ŷ, histo = lsqr(AN, b; atol=tol, btol=tol, itmax=itmax, history=true)

    x̂ = N * ŷ
    return x̂, histo
end